In [171]:
import pandas as pd
import numpy as np
import re
import os

In [172]:
path = os.path.abspath(r'C:\Users\gabri\Workplace\LabEstat\code\data\t')
list_archive = os.listdir(path)
list_frames = [0 for x in list_archive]
for index, archive in enumerate(list_archive):
    try:
        list_frames[index] = pd.read_csv(os.path.join(path,archive))
    except Exception:
        list_frames[index] = pd.read_csv(os.path.join(path,archive), encoding='latin1')


In [173]:
exl_ls = [0 for x in list_frames]
for index, i in enumerate(list_frames):
    a = list_frames[index].iloc[:,0]
    exl_ls[index] = list(list_frames[index][a.str.contains('Instituto')].index)
exl_ls

[[0, 8, 13, 17, 31, 33, 41, 50, 53, 63, 68, 78],
 [0, 4, 12, 16, 24, 35, 37, 48, 50, 55, 60, 69],
 [0, 5, 13, 16, 23, 32, 34, 42, 44, 51, 56, 64],
 [0, 5, 13, 16, 23, 33, 41, 43, 50, 55, 63],
 [0, 6, 16, 19, 27, 40, 42, 50, 52, 66, 71, 80],
 [0, 2, 8, 17, 20, 28, 40, 42, 56, 58, 68, 73, 83],
 [],
 [0, 2, 8, 17, 20, 27, 39, 41, 50, 53, 66, 71, 80],
 []]

In [174]:
year = 2014
dic={}
for index, i in enumerate(list_frames):
    try:
        list_frames[index] = i.drop(exl_ls[index])
    except KeyError:
        pass
    dic[year]= pd.Series([''.join(list(x)) for x in list_frames[index].iloc[:,0].str.extract(r'([A-Za-zçãâáéíâêôóúõ\s]+)').values])
    year+=1

In [175]:
cursos = []
for year in dic.keys():
    for curso in list(dic[year].unique()):
        if curso.strip() in cursos:
            continue
        else:
            cursos.append(curso.strip())

In [176]:
cursos

['Artes visuais',
 'Cinema e Audiovisual',
 'Dança',
 'Museologia',
 'Música',
 'Teatro',
 'Biomedicina',
 'Biotecnologia',
 'Ciências Biológicas',
 'Educação Física',
 'Pedagogia',
 'Ciência da Computação',
 'Ciências Naturais',
 'Estatística',
 'Física',
 'Matemática',
 'Química',
 'Química Industrial',
 'Sistemas de Informação',
 'Direito',
 'Enfermagem',
 'Farmácia',
 'Fisioterapia',
 'Medicina',
 'Nutrição',
 'Odontologia',
 'Terapia ocupacional',
 'Administração',
 'Arquivologia',
 'Arquitetura e urbanismo',
 'Biblioteconomia',
 'Ciências Contábeis',
 'Ciências Econômicas',
 'Serviço Social',
 'Turismo',
 'Licenciatura Integrada em Educação em Ciências Matemática e Linguagens',
 'Ciências Sociais',
 'Filosofia',
 'Geografia',
 'História',
 'Intervalar Psicologia',
 'Psicologia',
 'Geofísica',
 'Geologia',
 'Meteorologia',
 'Oceanografia',
 'Comunicação Social',
 'Letras',
 'Engenharia Civil',
 'Engenharia da Computação',
 'Engenharia de Alimentos',
 'Engenharia de Telecomunicaçõe

In [177]:
# Finding NaN values to revise

for t in range(8):
    print(t,'=>',len(list_frames[t].iloc[:,:][list_frames[t].iloc[:,-1].isna()].values),'-',end=' ')
    print(list_frames[t].iloc[:,:][list_frames[t].iloc[:,-1].isna()].values)

0 => 0 - []
1 => 0 - []
2 => 0 - []
3 => 0 - []
4 => 0 - []
5 => 0 - []
6 => 0 - []
7 => 0 - []


In [178]:
# Making the DataFrame

dic_cursos = {}
for curso in cursos:
    can=[0 for x in range(8)]
    # print(curso)
    for t in range(8):
        temp = sum([int(x[-1]) for x in list_frames[t][list_frames[t].iloc[:,0].str.contains(curso)].values])
        # print(temp,end=' ')
        can[t]=temp
    # print('-----------------------------------')
    dic_cursos[curso]=can
df_cursos = pd.DataFrame(dic_cursos, index=[x for x in range(2014,2022)])
df_cursos = df_cursos.drop(['Total geral','Total Geral','Total','Intervalar Psicologia'],axis=1)
df_cursos = df_cursos.T

In [196]:
# Corrections and Revisions

tags = {'História':['Historia','História Mosqueiro'],
'Artes Visuais': ['Artes visuais'],
'Terapia Ocupacional':['Terapia ocupacional'],
'Licenciatura Integrada em Educação em Ciências Matemática e Linguagens':['Licenciatura Integrada em Ciências Matemática e Linguagens','Licenciatura Integrada em educação'],
'Economia':['Ciências Econômicas'],
'Produção Multimídia':['Tecnologia em Produção Multimídia'],
'Arquitetura e Urbanismo':['Arquitetura e urbanismo'],
'Ciências Sociais':['Ciências Sociais Aplicadas']
}

for tag in tags:
    can=df_cursos.loc[tag]
    for t in tags[tag]:
        can+=df_cursos.loc[t]
        df_cursos = df_cursos.drop(t)
    df_cursos.loc[tag]=can

reps = {'Matemática':['Licenciatura Integrada em Educação em Ciências Matemática e Linguagens'],
'Física':['Educação Física'],
'Química':['Engenharia Química', 'Química Industrial'],
'Engenharia Sanitária':['Engenharia Sanitária e Ambiental']
}

for rep in reps:
    can=df_cursos.loc[rep]
    for t in reps[rep]:
        can-=df_cursos.loc[t]
    df_cursos.loc[rep]=can

df_cursos.loc['Licenciatura Integrada em Educação em Ciências Matemática e Linguagens'][2015]=21
df_cursos.loc['Matemática'][2015]=32
df_cursos.loc['Produção Multimídia'][2016]=1
df_cursos.loc['Produção Multimídia'][2017]=5
df_cursos.loc['Economia'][2017]=184
df_cursos.loc['Produção Multimídia'][2018]=df_cursos.loc['Produção Multimídia'][2018]/2
df_cursos.loc['Ciências Sociais'][2019]=29
df_cursos.loc['História'][2021]=21
df_cursos.loc['Letras'][2021]=55
df_cursos.loc['Nutrição'][2021]=12

In [197]:
with pd.option_context('display.max_rows',None,'display.max_columns',None):
    print(df_cursos.T.loc[2021])

Cinema e Audiovisual                                                       3
Dança                                                                      3
Museologia                                                                 2
Música                                                                     0
Teatro                                                                     6
Biomedicina                                                                6
Biotecnologia                                                             10
Ciências Biológicas                                                       16
Educação Física                                                            2
Pedagogia                                                                 16
Ciência da Computação                                                      4
Ciências Naturais                                                         18
Estatística                                                                8

In [184]:
for t in range(2021,2022):
    print('\n'.join([f'{x} - {y}' for (x,y) in zip(df_cursos[t].index,df_cursos[t])]))
    print('---------------------------------------------------------------')

Cinema e Audiovisual - 3
Dança - 3
Museologia - 2
Música - 0
Teatro - 6
Biomedicina - 6
Biotecnologia - 10
Ciências Biológicas - 16
Educação Física - 2
Pedagogia - 16
Ciência da Computação - 4
Ciências Naturais - 18
Estatística - 8
Física - 41
Matemática - 22
Química - 13
Química Industrial - 11
Sistemas de Informação - 6
Direito - 13
Enfermagem - 11
Farmácia - 14
Fisioterapia - 8
Medicina - 6
Nutrição - 0
Odontologia - 13
Administração - 21
Arquivologia - 6
Biblioteconomia - 14
Ciências Contábeis - 16
Serviço Social - 21
Turismo - 25
Licenciatura Integrada em Educação em Ciências Matemática e Linguagens - 7
Ciências Sociais - 22
Filosofia - 16
Geografia - 10
História - 23
Psicologia - 5
Geofísica - 19
Geologia - 12
Meteorologia - 16
Oceanografia - 6
Comunicação Social - 8
Letras - 57
Engenharia Civil - 23
Engenharia da Computação - 12
Engenharia de Alimentos - 8
Engenharia de Telecomunicações - 15
Engenharia Elétrica - 13
Engenharia Mecânica - 21
Engenharia Naval - 4
Engenharia Químic

In [187]:
df_cursos.to_csv(os.path.join(r'C:\Users\gabri\Workplace\LabEstat\code\data\out','datacursos.csv'), index=True)